### Тональность текста
В блокноте text_classification.ipynb разобрали анализ тональности используя полносвязанную сеть сделать на тех же данных
1. tf-idf/count vectorizer + логистическую регрессию
2. Обучить вашу архитектуре сети возможно туже что и была на занятии проверить
* взять предобученный эмбединг(к примеру word2vec) и загрузить в слой Embedding
* взять слой Embedding без предобученных весов
Сравнить все подходы в том числе и полносвязанную сеть что лучше отработало

#### 2. Проверяю модель, которая была на занятии

In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

84131840/84125825 [==============================] - 33s 0us/step


In [3]:
os.listdir(dataset_dir)

['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'train']

In [4]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [5]:
sample_file = os.path.join(train_dir, 'pos/1181_9.txt')
with open(sample_file) as f:
  print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


In [6]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [7]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [8]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to neg
Label 1 corresponds to pos


In [9]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [10]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.


In [11]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [12]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [13]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [14]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [15]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [16]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [17]:
embedding_dim = 16

In [19]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim,),
  #layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  #layers.Dropout(0.2),
  layers.Dense(145),
  layers.Dense(1)])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160016    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 145)               2465      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 146       
Total params: 162,627
Trainable params: 162,627
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [21]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
625/625 [==============================] - 36s 57ms/step - loss: 0.4597 - binary_accuracy: 0.7715 - val_loss: 0.3127 - val_binary_accuracy: 0.8700
Epoch 2/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2432 - binary_accuracy: 0.9035 - val_loss: 0.3011 - val_binary_accuracy: 0.8744
Epoch 3/10
625/625 [==============================] - 2s 4ms/step - loss: 0.1844 - binary_accuracy: 0.9309 - val_loss: 0.3289 - val_binary_accuracy: 0.8698
Epoch 4/10
625/625 [==============================] - 2s 4ms/step - loss: 0.1444 - binary_accuracy: 0.9487 - val_loss: 0.3768 - val_binary_accuracy: 0.8658
Epoch 5/10
625/625 [==============================] - 2s 4ms/step - loss: 0.1126 - binary_accuracy: 0.9627 - val_loss: 0.4411 - val_binary_accuracy: 0.8658
Epoch 6/10
625/625 [==============================] - 2s 4ms/step - loss: 0.0867 - binary_accuracy: 0.9732 - val_loss: 0.5250 - val_binary_accuracy: 0.8622
Epoch 7/10
625/625 [==============================] - 2s 4ms/s

In [22]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 [==============================] - 35s 44ms/step - loss: 1.3415 - binary_accuracy: 0.8037
Loss:  1.34149968624115
Accuracy:  0.8036800026893616


 #### 1. tf-idf/count vectorizer + логистическая регрессия

In [136]:
import pandas as pd
import os
import re
import html

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import linear_model, metrics

In [101]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [92]:
def create_dataset_from_path(path, label):
    file_names = os.listdir(path)
    # Create Dictionary for File Name and Text
    file_name_and_text = {}
    for file in file_names:
        with open(path + file, "r", encoding="utf8") as target_file:
            file_name_and_text[file] = target_file.read()
        
    df = (pd.DataFrame.from_dict(file_name_and_text, orient='index')
             .reset_index().rename(index = str, columns = {'index': 'file_name', 0: 'text'}))
    df['label'] = label
    return df

Собираем тренировочный датасет

In [93]:
train_neg = create_dataset_from_path("./aclImdb/train/neg/",0)
train_pos = create_dataset_from_path("./aclImdb/train/pos/",1)
train = pd.concat([train_pos, train_neg], ignore_index=True, axis=0)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
file_name    25000 non-null object
text         25000 non-null object
label        25000 non-null int64
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


Собираем тестовый датасет

In [94]:
test_neg = create_dataset_from_path("./aclImdb/test/neg/",0)
test_pos = create_dataset_from_path("./aclImdb/test/pos/",1)
test = pd.concat([train_pos, train_neg], ignore_index=True, axis=0)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
file_name    25000 non-null object
text         25000 non-null object
label        25000 non-null int64
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


Приведем все к нижнему регистру, удалим html разметку и знаки пунктуации

In [117]:
def clean_text(df, columntxt_name):
    df['clean_text'] = df[columntxt_name].str.replace('<br />', ' ')
    df['clean_text'] = df['clean_text'].apply(lambda x : html.unescape(x))
    df['clean_text'] = df['clean_text'].apply(lambda x : x.lower())
    df['clean_text'] = df['clean_text'].apply(lambda x : re.sub(r'[^\w\s]', '', x))    
    return df

In [118]:
train = clean_text(train,'text')
train.head(2)

,file_name,text,label,clean_text
0,0_9.txt,"Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as ""Teachers"". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is ""Teachers"". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that...",1,bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell highs satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector im here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isnt
1,10000_8.txt,"Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they'll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it's like to be homeless? That is Goddard Bolt's lesson.<br /><br />Mel Brooks (who ...",1,homelessness or houselessness as george carlin stated has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school work or vote for the matter most people think of the homeless as just a lost cause while worrying about things such as racism the war on iraq pressuring kids to succeed technology the elections inflation or worrying if theyll be next to end up on the streets but what if you were given a bet to live on the streets for a month without the luxuries you once had from a home the entertainment sets a bathroom pictures on the wall a computer and everything you once treasure to see what its like to be homeless that is goddard bolts lesson mel brooks who directs who stars as bolt plays a rich man ...


In [119]:
test = clean_text(test, 'text')
test.head(2)

,file_name,text,label,clean_text
0,0_9.txt,"Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as ""Teachers"". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is ""Teachers"". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that...",1,bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell highs satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector im here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isnt
1,10000_8.txt,"Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they'll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it's like to be homeless? That is Goddard Bolt's lesson.<br /><br />Mel Brooks (who ...",1,homelessness or houselessness as george carlin stated has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school work or vote for the matter most people think of the homeless as just a lost cause while worrying about things such as racism the war on iraq pressuring kids to succeed technology the elections inflation or worrying if theyll be next to end up on the streets but what if you were given a bet to live on the streets for a month without the luxuries you once had from a home the entertainment sets a bathroom pictures on the wall a computer and everything you once treasure to see what its like to be homeless that is goddard bolts lesson mel brooks who directs who stars as bolt plays a rich man ...


In [129]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=10000)
tfidf_vect.fit(train['clean_text'])
xtrain_tfidf =  tfidf_vect.transform(train['clean_text'])
xvalid_tfidf =  tfidf_vect.transform(test['clean_text'])

In [130]:
xtrain_tfidf.shape

(25000, 10000)

In [134]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, valid_y):
    classifier.fit(feature_vector_train, label)    
    predictions = classifier.predict(feature_vector_valid)
    
    return metrics.accuracy_score(predictions, valid_y)

In [137]:
# Linear Classifier на Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train['label'], xvalid_tfidf, test['label'])
print("LR, WordLevel TF-IDF: ", accuracy)

LR, WordLevel TF-IDF:  0.92192
